<a href="https://colab.research.google.com/github/JDS289/BaLD4LLM/blob/main/playingAround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using GPU seems to sometimes cause a CUDA error; using TPU seems to sometimes cause "Unknown crash".
CPU takes approximately 10x longer than GPU.


3B has said False to "Paris is the capital of France". Might want to use 8B instead. For now, I'll continue to use 3B for speed/simplicity. This is also justified for "reproducibility" reasons, though if accuracy is negligible then switch to a better model.

8B works with TPU? (sample size 1)
but still insanely slow - hasn't yet answered a single prompt

try 3B with a sys_prompt?

In [1]:
from google.colab import userdata
hf_token = userdata.get("huggingface_secret")

from huggingface_hub import login
login(token=hf_token)

In [2]:
DEFAULT_MODEL = ["meta-llama/Llama-3.1-8B-Instruct", "meta-llama/Llama-3.2-3B-Instruct"][1]

from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm

#import warnings
#warnings.filterwarnings('ignore')


# Display first 500 characters of extracted text as preview


device = "cuda" if torch.cuda.is_available() else "cpu"

#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SYS_PROMPT = "" # experiment with this?


#Let's load in the model and start processing the text chunks

accelerator = Accelerator()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="balanced",
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id


model-00001-of-00002.safetensors:  21%|##        | 1.03G/4.97G [00:00<?, ?B/s]

In [ ]:
"""model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="auto",  # Or try 'balanced' or 'balanced_low_0' for multi-GPU
    offload_folder="offload",  # Specify a folder for offloading model weights
    offload_state_dict=True,   # Enable offloading the state dict to disk
    offload_buffers=True,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id"""

In [ ]:
model.generation_config.output_hidden_states = True
model.generation_config.return_dict_in_generate = True
# print(model.generation_config)

In [ ]:
def process_chunk(text_chunk):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=1, # was 0.7 # maybe check this
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).strip()
    processed_text = processed_text[processed_text.index("assistant") + 11:]


    """
    for i in range(len(output.hidden_states)):
        total_shape = np.array(output.hidden_states[i][0].shape)
        for j in range(1, len(output.hidden_states[i])):
            total_shape += np.array(output.hidden_states[i][j].shape)
        print(total_shape)"""
    return processed_text, output.hidden_states

In [ ]:
import requests

capitals = requests.get("https://raw.githubusercontent.com/JDS289/BaLD4LLM/refs/heads/main/country-list.csv")
capitals_dict = {}
for string in capitals.content.decode('utf-8').split('\n')[1 : -1]:
  _, country, _, city, _ = string.split('"')
  capitals_dict[country] = city

print(capitals_dict)


In [ ]:
# trying to use fewer words in the prompt, hopefully for faster execution

true_list = [f"Is {capital} the capital of {country}?" for country, capital in capitals_dict.items()]
fixed_order_capitals_dict = list(capitals_dict.items())
false_list = [f"Is {fixed_order_capitals_dict[i][1]} the capital of {fixed_order_capitals_dict[(i+1) % len(capitals_dict)][0]}?" for i in range(len(capitals_dict))]
print(true_list[:10], false_list[:10])

def print_ret(x):
  print(x[:2])
  return x

results = [print_ret((q,) + process_chunk(f'{q} (Yes/No)')) for q in tqdm(true_list[:20])]
results_false = [print_ret((q,) + process_chunk(f'{q} (Yes/No)')) for q in tqdm(false_list[:20])]

statements = []
responses = []
hidden_statess = []

for s, r, h in results:
  statements.append(s)
  responses.append(r)
  hidden_statess.append(h)

In [ ]:
"""true_list = [f"{capital} is the capital city of {country}" for country, capital in capitals_dict.items()]
fixed_order_capitals_dict = list(capitals_dict.items())
false_list = [f"{fixed_order_capitals_dict[i][1]} is the capital city of {fixed_order_capitals_dict[(i+1) % len(capitals_dict)][0]}" for i in range(len(capitals_dict))]
print(false_list[:20])

def print_ret(x):
  print(x[:2])
  return x

results = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(true_list[:20])]
results_false = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(false_list[:20])]

statements = []
responses = []
hidden_statess = []

for s, r, h in results:
  statements.append(s)
  responses.append(r)
  hidden_statess.append(h)
"""

In [ ]:
process_chunk("hi")

In [ ]:
print(statements)

In [ ]:
# print(len(hidden_statess[0]))
print(hidden_statess[0][0])



# 7 * 29 * 60 * 3072  for EACH hidden_statess[i]

# 7 layers * 29 batches * 60 tokens per input * 3072 activations

# idk why there are 29 batches

In [ ]:
conversation = [
      {"role": "system", "content": SYS_PROMPT},
      {"role": "user", "content": f'"{statements[0]}". Is this true or false? (Answer ONLY with "True." or "False.")'},
]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [ ]:
print(inputs["input_ids"].shape)